In [1]:
%load_ext autoreload
%autoreload 2
import os
# os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# !export HF_HOME=/shared/data3/pk36/.cache
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"

In [3]:
from debate import DebateNode
from paper_details import Paper
from persona import PaperAuthor
from moderator import Moderator
import argparse
from typing import List
from vllm import LLM
import os
import json
from data_pairer import parse_papers
import numpy as np

/home/ishikaa2/miniconda3/envs/tod/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-29 12:15:45,730	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
class Args():
    def __init__(self):
        self.log_dir = "logs"
        self.focus_paper = "https://arxiv.org/pdf/2406.11709" # treeinstruct
        self.cited_paper = "https://arxiv.org/pdf/2310.10648" # bridge
        self.topic = {'argument_title': "helping students fix their errors", 'description': ""}

args = Args()

In [5]:
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

# if not os.path.exists("data.json"):
parse_papers(args.focus_paper, args.cited_paper)
with open('data.json', 'r') as file:
    data = json.load(file)

In [6]:
model_server = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",tensor_parallel_size=2, gpu_memory_utilization=0.5, max_num_seqs=100) #,enable_prefix_caching=True)

INFO 11-29 12:15:59 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-29 12:15:59 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-29 12:15:59 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-29 12:15:59 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.79it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.84it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.75it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.89it/s]



INFO 11-29 12:16:07 model_runner.py:1067] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=1653491) INFO 11-29 12:16:08 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-29 12:16:09 distributed_gpu_executor.py:57] # GPU blocks: 13892, # CPU blocks: 4096
INFO 11-29 12:16:09 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 1.70x
INFO 11-29 12:16:10 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=1653491) INFO 11-29 12:16:10 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=1653491) INFO 11-29 12:16:10 model_runner.py:1399] CUDA graphs can tak

In [7]:
item = data[0]
f_pap = item['focus']
c_pap = item['cited']

focus_paper = PaperAuthor(
    model = model_server,
    paper = Paper(f_pap),
    focus=True,
    id=0
)

cited_paper = PaperAuthor(
    model = model_server,
    paper = Paper(c_pap),
    focus=False,
    id=1
)

100%|██████████| 3/3 [00:01<00:00,  2.76it/s]


In [15]:
moderator = Moderator(model_server)

paper_authors = [focus_paper, cited_paper]
leaf_node_label = args.topic

if args.log_dir != "":
    with open(os.path.join(args.log_dir, 'self_deliberation.txt'), 'w') as f:
        f.write(f'Topic: {args.topic}\n\n')

# each node has a topic
root_node = DebateNode(leaf_node_label)
subtrees = root_node.conduct_self_deliberation(leaf_node_label, paper_authors, log=args.log_dir) # k new, finer topics to discuss

100%|██████████| 1/1 [00:00<00:00, 82.68it/s]


In [14]:
root_node.self_delib

{0: [{'argument_title': 'Our approach helps students learn from their mistakes through self-discovery',
   'description': "Our method guides students to identify and fix their own errors, promoting self-discovery and learning. This is supported by the Instructor's single-turn responses in the 'Help Fix Code' and 'Question from Code' modules, which direct students towards their mistakes and use natural language to describe bug fixes, as seen in Retrieved Evidence #1. Furthermore, the Instructor guides students to generate a list of all bug fixes based on their interactions, allowing students to resolve their own conceptual and syntactical errors in a Socratic fashion, as mentioned in Retrieved Evidence #2."},
  {'argument_title': 'Our approach helps students develop problem-solving skills through Socratic questioning',
   'description': 'Our method uses Socratic questions to guide students through the problem-solving process, helping them develop critical thinking and problem-solving sk

In [10]:
root_node.preemption

{0: {'Does my paper also address the claim, "bidirectional language model architecture"?': ['We used a beam size of 21 and a = 0.3 for both WSJ only and the semi-supervised setting. Our results in Table 4 show that despite the lack of task-specific tuning our model performs surprisingly well, yielding better results than all previously reported models with the exception of the Recurrent Neural Network Grammar [8]',
   'Introduction Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]'],
  'Does my paper also address the claim, "advancements in pre-training objectives"?': ['During inference, we Table 4: The Transformer generalizes

In [11]:
root_node.children

[Transformer Architecture Offers Superior Performance,
 Transformer Architecture Overcomes Recurrent Model Limitations]

In [16]:
convo_history = root_node.children[0].conduct_debate(paper_authors)


PRESENT ARGUMENT FOR AUTHOR 0:



Compiling FSM index for all state transitions: 100%|██████████| 57/57 [00:02<00:00, 23.99it/s]


You are an author of a paper that is debating another author on your claim topic:
	- Topic: Our approach guides students to resolve errors through Socratic questioning
	- Topic Description: Our method encourages students to think critically and understand the underlying concepts, rather than simply providing direct code answers. This is supported by Evidence #1, which shows that our Instructor provides single-turn responses that direct the student towards where their mistake is and uses natural language to describe the bug fixes, allowing students to learn from their errors.

Your debate claim is that your paper's contributions towards the topic are all novel relative to the other paper. Refer to their arguments and presented evidence, as well as your own paper's segments as evidence when refining your arguments.

You used the following evidence to support your arguments:
	- Evidence #1. In their "Help Fix Code" and "Question from Code" modules, the Instructor provides single-turn resp

In [17]:
print(convo_history)

Debate Topic Information:
	- Topic: Our approach guides students to resolve errors through Socratic questioning
	- Topic Description: Our method encourages students to think critically and understand the underlying concepts, rather than simply providing direct code answers. This is supported by Evidence #1, which shows that our Instructor provides single-turn responses that direct the student towards where their mistake is and uses natural language to describe the bug fixes, allowing students to learn from their errors.

Debate History:

	-Author 0: I argue that novelty in socratic questioning approach. Our paper's contributions are novel due to the unique combination of Socratic questioning and LLM-based tutoring, which steers students towards resolving errors through critical thinking. Unlike the opposing paper, our method does not rely on direct code answers or error categorization systems, but rather focuses on guiding students to understand their conceptual and syntactical mistake

In [15]:
root_node.children[0].response[0]

{'argument_list': [{'title': 'Argument 1',
   'description': 'Argument 1 description'},
  {'title': 'Argument 2', 'description': 'Argument 2 description'}]}

In [ ]:
conversation_history = []

queue_of_rounds: List[DebateNode] = []
queue_of_rounds.extend(subtrees)


while len(queue_of_rounds) > 0:
    round = queue_of_rounds.pop(0)
    conversation, new_focus_arg, new_cited_arg = round.conduct_debate(focus_paper, cited_paper)
    conversation_history.extend(conversation)
    if moderator.is_expand(round.self_delib, [new_focus_arg, new_cited_arg]):
        new_subtrees = round.conduct_self_deliberation(round.round_topic, paper_authors)
        queue_of_rounds.extend(new_subtrees)

with open('conversation_history.txt', 'w+') as f:
    f.write('\n'.join(conversation_history))